In [1]:
import pandas as pd
import numpy as np
import math
import pickle
import random

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat
from scipy.spatial import distance

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering

from src import SMOTE
from src import CFS
from src import metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

In [120]:
def load_data(project,commits,file_commits):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix', 'contains_bug','fixes',],axis=1)
    
#     commit_guru_df = commit_guru_df[commit_guru_df['commit_hash'].isin(commits)]
    understand_df['file'] = [0]*understand_df.shape[0]
    for i in range(understand_df.shape[0]):
        understand_df.loc[i,'file'] = understand_df.loc[i,'Name'].split('.')[-1]
    sub_df = pd.DataFrame()
    for commit in file_commits:
        _df = understand_df[understand_df['commit_hash'] == commit]
        _df = _df[_df['file'].isin(file_commits[commit]['files'])]
        sub_df = pd.concat([sub_df,_df],axis = 0)
#     print(sub_df)
    understand_df = sub_df
#     print(understand_df.shape)
#     understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
#     print(understand_df.shape)
    df = understand_df.merge(commit_guru_df,on='commit_hash')
#     print(df.shape)
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash','file']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y

def load_understand_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
#     print(cols_list)
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind', 'Name']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def load_commit_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    commit_guru_df = pd.read_csv(commit_guru_path)
    
    commit_guru_df = commit_guru_df.drop(labels = ['author_name','author_name',
                                                   'author_email','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix','fixes',],axis=1)

    df = commit_guru_df
    df.rename(columns={"contains_bug": "Bugs"},inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    y.fillna(False,inplace=True)
    df['Bugs'] = y
    df.dropna(inplace=True)
    cols = df.columns.tolist()
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def normalize(X):
    commit_hash = X.commit_hash
    X = X.drop('commit_hash',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    X['commit_hash'] = commit_hash
    return X

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
    y = df.Bugs.values
    X = df.drop(labels = ['Bugs'],axis = 1)
    X = X.values
    selected_cols = CFS.cfs(X,y)
    cols = df.columns[[selected_cols]].tolist()
    cols.append('Bugs')
    return df[cols],cols

In [121]:
def get_commit_subset_s2(project):
    commit_df = load_commit_data(project)
    commits = []
    file_commits = {}
    updated_commits = []
    cols = commit_df.columns.values.tolist()
    cols.append('file')
    commit_df = commit_df[commit_df['Bugs'] == True]
    commit_df.reset_index(drop=True,inplace=True)
    for i in range(commit_df.shape[0]):
        files = commit_df.loc[i,'fileschanged'].split(',')
        cleaned_files = [file.split('/')[-1].split('.')[0] for file in files]
        parent_hashes = commit_df.loc[i,'parent_hashes']
        file_commits[commit_df.loc[i,'commit_hash']] = {'files':cleaned_files,'parent':parent_hashes}
        for file in files:
            if file.split('.')[-1] == 'java':
                commit_info = commit_df.loc[i].values.tolist()
                commit_info.append(file)
                updated_commits.append(commit_info)
    updated_commits_df = pd.DataFrame(updated_commits,columns = cols)
    updated_commits_df.drop('fileschanged',axis = 1,inplace = True)
    unique_files = updated_commits_df.file.unique()
    commits.append(commit_df.commit_hash.values.tolist()) 
    commits = [val for sublist in commits for val in sublist]
    return commits,file_commits


In [122]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [123]:
def run(project):
    commits,file_commits = get_commit_subset_s2(project)
    X,y = load_data(project,commits,file_commits)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc

In [128]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
count = 0
for project in projects:
    if project != 'JustAuth':
        continue
    try:
        recall,precision,pf,f1,g_score,auc = run(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.98      1.00      0.99        47

    accuracy                           0.98        48
   macro avg       0.49      0.50      0.49        48
weighted avg       0.96      0.98      0.97        48

